In [1]:
import pandas as pd
import sys, torch
# sys.path.append("/Users/tracy/Library/CloudStorage/GoogleDrive-cloudstorage.yuzhe@gmail.com/My Drive/UPENN♥️/MyClasses/23Fall/CIS5190/project/nlp")

# from traditional.features import craft_features, vectorize_labels, FEAT_ARG
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

from deep.rnn_classifier import RNNBinarySequenceClassifier

import warnings
warnings.filterwarnings("ignore")

### Domain Shift for Traditional Model

For this one, we evaluate two traditional models.

One is n-gram tfidf only model; The other is n-gram + lexicon model

In [2]:
# train = pd.read_csv("./data/imdb/train_cleaned.csv")
# val = pd.read_csv("./data/imdb/val_cleaned.csv")
test = pd.read_csv("./data/twitter/test_cleaned.csv")

# train_texts = [text for text in train["text_cleaned"]]
test_texts = [text for text in test["text_cleaned"]]
# val_texts = [text for text in val["text_cleaned"]]

# train_labels = [senti for senti in train["label"]]
test_labels = [senti for senti in test["label"]]
# val_labels = [senti for senti in val["label"]]

# splitted_texts, splitted_labels = {"train": train_texts, "test": test_texts, "val": val_texts},  {"train": train_labels, "test": test_labels, "val": val_labels}

In [ ]:
NGRAM_RANGE = (1,3)
MAX_TFIDF_FEATS = 5000
MIN_DF = 3
MAX_DF = 0.7

print("Without Lexicon Model with N=5000")

args = FEAT_ARG(NGRAM_RANGE, MIN_DF, MAX_DF, MAX_TFIDF_FEATS)
FEATURESET = "tfidf"
X_train, X_val, X_test = craft_features(featset=FEATURESET, text_splits=splitted_texts, feat_args=args)
y_train, y_val, y_test = vectorize_labels(splitted_labels)
print("Features:  Train {} , Val {} , Test {}".format(X_train.shape, X_val.shape, X_test.shape))

# These are the best parameters
p = 'l2'
lambda_ = 1.

lr = LogisticRegression(C=1/lambda_, penalty=p, solver="liblinear", max_iter=5000)
lr.fit(X_train, y_train)
train_pred = lr.predict(X_train)
val_pred = lr.predict(X_val)

test_pred = lr.predict(X_test)
print("Trainset")
print(classification_report(y_train, train_pred))
print("Valset")
print(classification_report(y_val, val_pred))
print("Testset (Domain shift)")
print(classification_report(y_test, test_pred))

Without Lexicon Model with N=5000
Load a pre-trained vectorizer: tfidf_vectorizer_ngram(1, 3)_max_5000_dfminmax_3_0.7.pickle
Features:  Train (20000, 5000) , Val (5000, 5000) , Test (61998, 5000)
Trainset
              precision    recall  f1-score   support

           0       0.92      0.91      0.91     10000
           1       0.91      0.92      0.92     10000

    accuracy                           0.92     20000
   macro avg       0.92      0.92      0.92     20000
weighted avg       0.92      0.92      0.92     20000

Valset
              precision    recall  f1-score   support

           0       0.89      0.86      0.88      2500
           1       0.87      0.89      0.88      2500

    accuracy                           0.88      5000
   macro avg       0.88      0.88      0.88      5000
weighted avg       0.88      0.88      0.88      5000

Testset (Domain shift)
              precision    recall  f1-score   support

           0       0.60      0.60      0.60     30969
  

In [ ]:
NGRAM_RANGE = (1,3)
MAX_TFIDF_FEATS = 5000
MIN_DF = 3
MAX_DF = 0.7

print("With Lexicon Model with N=5000")

args = FEAT_ARG(NGRAM_RANGE, MIN_DF, MAX_DF, MAX_TFIDF_FEATS)
FEATURESET = "tfidf+lexicon"
X_train, X_val, X_test = craft_features(featset=FEATURESET, text_splits=splitted_texts, feat_args=args)
y_train, y_val, y_test = vectorize_labels(splitted_labels)
print("Features:  Train {} , Val {} , Test {}".format(X_train.shape, X_val.shape, X_test.shape))

# These are the best parameters
p = 'l2'
lambda_ = 1.

lr = LogisticRegression(C=1/lambda_, penalty=p, solver="liblinear", max_iter=5000)
lr.fit(X_train, y_train)
train_pred = lr.predict(X_train)
val_pred = lr.predict(X_val)

test_pred = lr.predict(X_test)
print("Trainset")
print(classification_report(y_train, train_pred))
print("Valset")
print(classification_report(y_val, val_pred))
print("Testset (Domain shift)")
print(classification_report(y_test, test_pred))

With Lexicon Model with N=5000
Load a pre-trained vectorizer: tfidf_vectorizer_ngram(1, 3)_max_5000_dfminmax_3_0.7.pickle
Load a pre-trained vectorizer: count_vectorizer_ngram(1, 3)_max_None_dfminmax_3_0.7.pickle
Retrieved Sentiment Lexicon with length 6786
Found 4372/6786 lexemes in training vocabulary
Found 4372/6786 lexemes in training vocabulary
Found 4372/6786 lexemes in training vocabulary
Features:  Train (20000, 11786) , Val (5000, 11786) , Test (61998, 11786)
Trainset
              precision    recall  f1-score   support

           0       0.95      0.95      0.95     10000
           1       0.95      0.95      0.95     10000

    accuracy                           0.95     20000
   macro avg       0.95      0.95      0.95     20000
weighted avg       0.95      0.95      0.95     20000

Valset
              precision    recall  f1-score   support

           0       0.88      0.86      0.87      2500
           1       0.87      0.88      0.87      2500

    accuracy        

### Domain Shift for Deep Learning Model

For this one, we evaluate two rnn models

One is Bi-GRU model by GloVe Learnable Embedding; Another is Bi-GRU model by BERT representation

In [3]:
from transformers import BertTokenizer
from datasets import Dataset
from tokenizers import Tokenizer
import torch.utils.data as torch_data

tokenizer_pth = "./deep/imdb50_tokenizer"
tokenizer = Tokenizer.from_file(tokenizer_pth)
orig_vocab = tokenizer.get_vocab()
word_types = sorted(list(orig_vocab.keys()), key=lambda w: orig_vocab[w])
vocab = {w: i for i, w in enumerate(word_types)}
vocab_size = len(vocab)
pad_id = vocab["<pad>"]

bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

class IMDB50(torch_data.Dataset):
    def __init__(self,
                 text,
                 labels,
                 tokenizer,):

        self.all_text = text
        self.all_labels = labels
        self.tokenizer = tokenizer

        self.is_bert = isinstance(tokenizer, BertTokenizer)

    def __len__(self):
        return len(self.all_text)

    def __getitem__(self, idx):

        if not self.is_bert:
            input_ids = torch.LongTensor(self.tokenizer.encode(self.all_text[idx]).ids)
        else:
            input_ids = self.tokenizer(self.all_text[idx], return_tensors='pt', max_length=512,
                                       padding="do_not_pad", truncation=True).input_ids.squeeze(0)

        label = torch.Tensor([self.all_labels[idx]])
        return input_ids, input_ids.size(0), label

import torch.nn.utils.rnn as rnn_utils
def collate_fn(batch):
    # input_id, length, label
    batch.sort(key=lambda x: x[1], reverse=True) # sort by sequence length
    sequences, seq_lengths, targets = zip(*batch)

    # Pad the sequences and stack the targets
    sequences_padded = rnn_utils.pad_sequence(sequences, padding_value=pad_id, batch_first=True)
    targets_stacked = torch.stack(targets)

    return sequences_padded, seq_lengths, targets_stacked

batch_size = 64
dataset = {"test": Dataset.from_pandas(test) }

test_text, test_label = dataset["test"]["text_cleaned"], dataset["test"]["label"]
testset = IMDB50(test_text, test_label, tokenizer)
testset_bert = IMDB50(test_text, test_label, bert_tokenizer)
test_loader = torch_data.DataLoader(testset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)
test_loader_bert = torch_data.DataLoader(testset_bert, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

print(next(iter(test_loader))[0].shape, next(iter(test_loader))[1], next(iter(test_loader))[2].shape)
print(next(iter(test_loader_bert))[0].shape, next(iter(test_loader_bert))[1], next(iter(test_loader_bert))[2].shape)

torch.Size([64, 30]) (30, 28, 28, 27, 26, 25, 24, 24, 24, 24, 23, 23, 23, 22, 21, 20, 19, 19, 18, 18, 18, 17, 16, 16, 16, 16, 16, 15, 15, 13, 13, 13, 13, 12, 12, 11, 11, 11, 11, 10, 10, 9, 9, 9, 8, 8, 8, 8, 8, 8, 7, 7, 6, 6, 6, 6, 5, 5, 4, 4, 4, 4, 4, 3) torch.Size([64, 1])
torch.Size([64, 40]) (40, 36, 35, 34, 33, 32, 32, 32, 31, 30, 30, 29, 28, 27, 27, 27, 27, 27, 25, 25, 25, 25, 24, 23, 21, 21, 20, 19, 19, 19, 19, 19, 18, 18, 18, 17, 17, 16, 16, 15, 15, 15, 15, 14, 14, 14, 14, 13, 13, 13, 13, 13, 12, 12, 12, 11, 11, 10, 10, 9, 8, 8, 7, 7) torch.Size([64, 1])


In [4]:
from tqdm import tqdm
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [7]:
model_pth = "./gru_rnn_glove_learnable_embedding_best.pt"
model1 = RNNBinarySequenceClassifier(
        vocab_size=vocab_size, embedding_size=256, hidden_size=256, output_size=1,
        num_layers=2, embedding_dropout=.3, output_dropout=.3, rnn_dropout=.3,
        rnn_base_cell="gru", embedding_type="glove", learnable=True, bidirectional=True, vocab=vocab
)
model1.load_state_dict(torch.load(model_pth))
model1 = model1.to(device)

model1.eval()
with torch.no_grad():
    pred_labels, true_labels = [], []
    for i, (input_ids, lengths, labels) in tqdm(enumerate(test_loader)):
        input_ids, labels = input_ids.to(device), labels.to(device)
        preds = model1.predict((input_ids, lengths))

        pred_labels.extend(preds.squeeze(-1).tolist())
        true_labels.extend(labels.squeeze(-1).tolist())

    acc = accuracy_score(true_labels, pred_labels)
    print(classification_report(true_labels, pred_labels))
    print("Accuracy: ", round(acc, 6))

.vector_cache/glove.6B.zip: 862MB [02:40, 5.37MB/s]                           
100%|█████████▉| 399999/400000 [00:52<00:00, 7594.59it/s]


Initialize by GLoVE word embedding


969it [00:06, 147.27it/s]


              precision    recall  f1-score   support

         0.0       0.61      0.47      0.53     30969
         1.0       0.57      0.70      0.63     31029

    accuracy                           0.58     61998
   macro avg       0.59      0.58      0.58     61998
weighted avg       0.59      0.58      0.58     61998

Accuracy:  0.582648


In [6]:
model_pth = "./gru_rnn_bert_representation_best.pt"
model2 = RNNBinarySequenceClassifier(
        vocab_size=vocab_size, embedding_size=768, hidden_size=256, output_size=1,
        num_layers=2, embedding_dropout=.3, output_dropout=.3, rnn_dropout=.3,
        rnn_base_cell="gru", embedding_type="bert", learnable=False, bidirectional=True, vocab=vocab,
)

model2.load_state_dict(torch.load(model_pth), strict=False)
model2 = model2.to(device)

model2.eval()
with torch.no_grad():
    pred_labels, true_labels = [], []
    for i, (input_ids, lengths, labels) in tqdm(enumerate(test_loader_bert)):
        input_ids, labels = input_ids.to(device), labels.to(device)
        preds = model2.predict((input_ids, lengths))

        pred_labels.extend(preds.squeeze(-1).tolist())
        true_labels.extend(labels.squeeze(-1).tolist())

    acc = accuracy_score(true_labels, pred_labels)
    print(classification_report(true_labels, pred_labels))
    print("Accuracy: ", round(acc, 6))

Use BERT representation [fixed]
BERT layers are freezed


969it [02:47,  5.79it/s]


              precision    recall  f1-score   support

         0.0       0.71      0.56      0.62     30969
         1.0       0.64      0.77      0.69     31029

    accuracy                           0.66     61998
   macro avg       0.67      0.66      0.66     61998
weighted avg       0.67      0.66      0.66     61998

Accuracy:  0.663038
